## QuerySet API Advanced

### CRUD 기본
- 모든 user 레코드 조회 : User.objects.all()
- user 레코드 생성 : User.objects.create(first_name='길동', last_name='홍', age=100, country='제주도', phone='010-1234-4567', balance=10000,)
- 101번 user 레코드 조회 : User.objects.get(pk=101)
- 101번 user 레코드의 last_name을 김으로 수정: user = User.objects.get(pk=101)
  - user.last_name = '김'
  - user.save()
- 101번 user 레코드 삭제 : user = User.objects.get(pk=101)
  - user.delete()
- 전체 인원 수 조회 : User.objects.count(), len(User.objects.all())

### Sorting data
- 나이가 어린 순으로 이름과 나이 조회하기
  - User.objects.order_by('age').values('first_name', 'age')
    - order_by(): 기본적으로 오름차순이며 필드명에 '-' 작성시 내림차순 정렬, 인자로 '?'를 입력하면 랜덤으로 정렬한다.
      - order_by()안에 인자를 두 개 쓰는 것이 아닌 order_by 자체를 두 번 쓰면 가장 마지막것만 적용이 된다.
    - values(): 모델 인스턴스가 아닌 딕셔너리 요소들을 가진 QuerySet을 반환
      - 조회하고자 하는 필드명을 가변인자로 입력받으며 각 딕셔너리에는 지정한 필드에 대한 key와 value만을 출력
      - 입력하지 않을 경우 각 딕셔너리에는 레코드의 모든 필드에 대한 key와 value를 출력

### Filtering data
- 중복없이 모든 지역 조회하기: User.objects.distinct().values('country')
- 지역 순으로 오름차순 정렬하여 중복없이 모든 지역 조회: User.objects.distinct().values('country').order_by('country')
- 나이가 30인 사람들의 이름 조회 : User.objects.filter(age=30).values('first_name')
- 나이가 30살 이상인 사람들의 이름과 나이 조회하기: User.objects.filter(age__gte=30).values('first_name', 'age')
  - Field lookups
    - SQL WHERE절의 상세한 조건을 지정하는 방법
    - QuerySet 메서드 filter, exclude, get에 대한 키워드 인자로 사용됨
    - field__lookuptype=value
    - 장고 필드 룩업에 대한 문서를 참고
- 나이가 30살 이상이고 계좌잔고가 50만원 초과인 사람들의 이름, 나이, 계좌 잔고 조회 : User.objects.filter(age__gte=30, balance__gt=500000).values('first_name', 'age', 'balance')
- 이름에 '호'가 포함되는 사람들의 이름과 성 조회하기: User.objects.filter(first_name__contains='호').values('first_name', 'last_name')
- 핸드폰번호가 011로 시작하는 사람들의 이름과 핸드폰번호 조회: User.objects.filter(phone__startswith='011-').values('first_name', 'phone')
  - SQL에서의 % 와일드카드와 같음
  - '_' 는 별도로 정규 표현식을 사용해야 함
- 이름이 '준'으로 끝나는 사람들의 이름 조회하기 : User.objects.filter(first_name__endswith='준').values('first_name')
- 경기도 혹은 강원도에 사는 사람들의 이름과 지역 조회: User.objects.filter(country__in=['경기도', '강원도']).values('first_name', 'country')
- 경기도 혹은 강원도에 살지 않는 사람들의 이름과 지역 조회: User.objects.exclude(country__in=['경기도', '강원도']).values('first_name', 'country')
- 나이가 가장 어린 10명의 이름과 나이 조회하기: User.objects.order_by('age').values('first_name', 'age')[:10]
- 나이가 30이거나 성이 김씨인 사람들 조회: User.objects.filter(Q(age=30) | Q(last_name='김'))
  - 단, 뷰함수에서는 from django.db.models import Q 해줘야 함
  - '&' 및 '|' 을 사용하여 Q 객체 결합 가능
  - 조회를 하면서 여러 Q객체를 제공할 수 있음
  - shell_plus에서는 import문 생략 가능

## Aggregation(Grouping data)

### aggregate()
- 전체 queryset에 대한 값을 계산
- 특정 필드 전체의 합, 평균, 개수 등을 계산할 때 사용
- 딕셔너리를 반환
- Avg, Count, Max, Min, Sum 등

### Aggregation 사용
- 나이가 30살 이상인 사람들의 평균 나이 조회하기
  - from django.db.models import Avg
  - User.objects.filter(age__gte=30).aggregate(Avg('age'))
  - 딕셔너리 이름 수정 가능 예) aggregate(ave_value=Avg('age'))
- 가장 높은 계좌 잔액 조회하기
  - from django.db.models import Max
  - User.objects.aggregate(Max('balance'))

### annotate()
- 쿼리의 각 항목에 대한 요약 값을 계산
- SQL의 GROUP BY에 해당
- '주석을 달다'라는 사전적 의미를 가지고 있음

### annotate 사용
- 각 성씨가 몇 명씩 있는지 조회
  - User.objects.values('last_name').annotate(Count('last_name'))
- 각 지역별로 몇 명씩 살고 있는지 조회하기
  - from django.db.models import Count
  - User.objects.values('country').annotate(Count('country'))
  - 딕셔너리 key값 수정 가능 예) annotate(num_of_fountry=Count('country'))
- 각 지역별로 몇 명씩 살고 있는지 + 지역별 계좌 잔액 평균 조회하기
  - User.objects.values('country').annotate(Count('country'), avg_balance=Avg('balance'))

### N:1 예시
- 만약 Comment - Article 관계가 N:1인 경우 다음과 같은 참조도 가능
``` python
Article.objects.annotate(
  number_of_comment=Count('comment'), # comment는 모델 이름
  pub_date=Count('comment', filter=Q(comment__created_at__lte='2000-01-01'))
)
```
- 전체 게시글을 조회하면서(Article.objects.all())
- annotate로 각 게시글의 댓글 개수(number_of_comment)와 2000-01-01보다 나중에 작성된 댓글의 개수(pub_date)를 함께 조회하는 것